In [1]:
import pandas as pd
import numpy as np
import nltk
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
import random

# Given dataset
extended_test_data = {
    'text_combined': [
        # Non-Phishing
        "Hi John, just wanted to remind you about our meeting tomorrow at 2 PM. Let me know if you need to reschedule.",
        "Dear Customer, your subscription will expire in 7 days. Please renew it to continue using our services. Thank you!",
        "We appreciate your recent purchase at our store! Please leave a review and let us know how we did.",
        "Congratulations! Your annual membership renewal was successful. Your account remains active until next year.",
        "Hi Sarah, here's your receipt for the tickets you purchased. Hope you enjoy the concert!",
        "Dear user, your application for the study program has been accepted. Please check your email for further details.",
        "Thank you for your interest in our property. We will get back to you shortly with available dates for viewing.",
        "Your Amazon package has been shipped! You can track it using the link provided in your account.",
        "Reminder: Your dental appointment is scheduled for 10:30 AM this Thursday. Please confirm your attendance.",
        "Hello! Your phone bill payment of $45 has been received. Thank you for choosing us.",
        "Hi, we have processed your refund of $30. Please allow 3-5 business days for it to reflect in your account.",
        "We’re writing to inform you that your flight to New York has been confirmed. Please review your itinerary.",
        "Your payment for order #1234 has been successfully processed. Delivery will be made within 5 business days.",
        "Welcome to our gym! We’re excited to have you. Here's your membership card to start your fitness journey.",
        "Dear customer, thank you for choosing our insurance. Your policy has been successfully renewed.",
        "Hello John, here are the minutes from last week's team meeting. Let me know if you need any clarification.",
        "This is to confirm your order for the iPhone 14. You will be notified once it ships.",
        "Your subscription to our newsletter has been confirmed. Stay tuned for the latest updates!",
        "Good news! Your job application has been moved to the interview stage. We will contact you soon.",
        "This is a confirmation email for your booking at the XYZ hotel. We look forward to your stay.",
        # Phishing
        "Your account has been compromised. Click here to reset your password immediately.",
        "Urgent! Your bank account has been locked. Click here to verify your details and regain access.",
        "You’ve been selected for a $1,000 gift card. Click here to claim your prize!",
        "Your PayPal account is under review. Please click here to update your information now.",
        "Final warning! Your Apple ID has been breached. Secure your account by clicking here.",
        "Congratulations! You’ve been randomly selected for a chance to win an iPhone 14. Claim now!",
        "We’ve detected unusual activity on your credit card. Please verify your information immediately.",
        "Your Netflix account will be suspended unless you update your payment information. Click here.",
        "You are entitled to a tax refund of $500. Click here to submit your bank details.",
        "Your mobile service will be disconnected unless you pay your overdue bill immediately. Click here to resolve.",
        "You are the lucky winner of a trip to Hawaii! Redeem your prize by clicking here now!",
        "Your Amazon account has been flagged for suspicious activity. Please verify your details immediately.",
        "We’ve detected an unusual login attempt from an unrecognized device. Click here to secure your account.",
        "Your insurance payment failed. Click here to update your billing details to avoid cancellation.",
        "Dear customer, click here to download your latest bank statement. Urgent action required.",
        "We’ve frozen your account due to suspicious transactions. Click here to reactivate your account.",
        "Your email storage is almost full! Upgrade now to avoid losing important emails.",
        "You've been pre-approved for a loan! Click here to accept your loan and transfer funds today.",
        "You’ve received a payment of $500. Click here to confirm your bank account to receive funds.",
        "Your Gmail account has exceeded its storage limit. Click here to upgrade for free.",
        "Congratulations! You've been selected for a free vacation. Confirm your booking here.",
        "You need to reset your password now to prevent your account from being locked. Click here.",
        "Your social media account is at risk. Click here to secure it before it gets hacked.",
        "Dear customer, you have been awarded a $100 gift card. Click here to claim it now.",
        "Important: We’ve detected fraudulent transactions on your card. Click here to review and secure your account.",
        "You have unpaid bills. Avoid penalties by making an immediate payment via this link.",
        "We’ve noticed suspicious login attempts on your account. Click here to verify your identity.",
        "Your subscription is about to expire. Click here to update your payment information now.",
        "Final warning: Your account will be deleted unless you verify your identity. Click here now.",
        "Your credit card has been flagged for fraudulent activity. Confirm your details here to resolve the issue.",
        "You’ve won a prize in our exclusive online lottery! Click here to claim your winnings now!",
        # More Non-Phishing
        "Dear student, your course materials are available for download on the university portal.",
        "Thank you for subscribing to our premium newsletter. You’ll receive exclusive updates every week.",
        "The concert you booked has been postponed to next Saturday. Please confirm if you can still attend.",
        "Hello! Your travel insurance policy has been renewed. Safe travels!",
        "Your electricity bill for the month of September is now available. Please review and make a payment.",
        "Your job application for the role of Software Engineer has been received. We’ll be in touch soon.",
        "Your account balance of $100 will be credited by the end of the day. Please check your bank statement.",
        "We’ve successfully updated your contact information as requested. Thank you for keeping your details up-to-date.",
        "You’ve earned 50 reward points! Redeem them in your next purchase.",
        "Dear user, your password has been successfully updated. Please contact support if this wasn’t you.",
        "Your next delivery has been scheduled for Monday, 8 AM. Track your package in the app.",
        "Hi! Your car service is scheduled for tomorrow. Please bring your car by 9 AM.",
        "Your tax return was processed successfully. Expect your refund within 3-5 business days.",
        "Dear tenant, your rent payment is due on the 1st of next month. Thank you.",
        "We’ve shipped your order! You can track the progress on our website.",
        "Thank you for choosing us for your banking needs. Your new card will be mailed to you soon.",
        "Your new account has been successfully created. Welcome to our banking family!",
        "We’re happy to inform you that your refund has been processed. Please allow a few days for the transaction.",
        "Dear customer, thank you for your recent feedback. We’ll strive to improve our services based on your suggestions.",
        "Your subscription to our service has been confirmed. You’ll now have access to premium features.",
        "Dear Customer, your invoice for the recent purchase is now available. You can download it from your account portal. Please contact support if you have any questions.",
        "Hi Mark, I’m confirming our meeting scheduled for next Tuesday at 10 AM. Let me know if you need to reschedule. Looking forward to it!",
        "We’re excited to introduce a new feature in our app! Update now to enjoy enhanced performance and new tools that help you stay productive.",
        " Dear Member, your gym membership will expire soon. Renew now to continue your fitness journey without interruption.",
        "You are invited to our annual customer appreciation event next month. Please RSVP by replying to this email. We look forward to seeing you there!",
        "Your payment of $100 has been received and processed successfully. Thank you for your prompt payment.",
        "Our office will be closed from December 24th to January 2nd in observance of the holiday season. We’ll resume normal operations on January 3rd. Happy Holidays!",
        "Your subscription is about to expire. Renew it today to continue enjoying premium content and uninterrupted service.",
        "Your password was successfully changed. If this wasn’t you, please contact support immediately.",
        "We value your feedback! Please take a moment to share your experience with our service. Your input helps us improve."

        # Phishing
        "Urgent! We have detected suspicious activity on your account. Click here to verify your identity and avoid account suspension.",
        "Congratulations! You have been selected to receive a brand-new iPhone 14. Click here to claim your prize now!",
        "Your recent payment could not be processed. Click here to update your payment information and avoid service disruption.",
        "You are the lucky winner of a $500 gift card. Click here to claim your reward before it expires.",
        "We’ve noticed unusual activity on your credit card. Please verify your account immediately by clicking this link.",
        "Your PayPal account has been suspended due to unusual activity. Click here to restore access to your account now",
        "Final warning! Your email account will be deleted in 24 hours unless you verify your details. Click here to prevent this.",
        "We need you to update your banking details to ensure you continue to receive payments. Click here to update your information securely",
        "Your Netflix account is at risk of suspension due to failed payment. Click here to update your billing details.",
        "You’ve been chosen for a free vacation! Click here to book your trip before the offer expire"
    ],
    'true_labels': [
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1
    ]
}

# Zipping the data together for shuffling
combined_data = list(zip(extended_test_data['text_combined'], extended_test_data['true_labels']))

# Shuffling the data
random.shuffle(combined_data)

# Unzipping the shuffled data
shuffled_text_combined, shuffled_true_labels = zip(*combined_data)

# Updated shuffled dataset
shuffled_data = {
    'text_combined': shuffled_text_combined,
    'true_labels': shuffled_true_labels
}

df = pd.DataFrame(shuffled_data)

In [3]:
import re
nltk.download('punkt', force=True)

# Data Preprocessing
# Each word in the text data is converted to lowercase and tokenized into individual words.
tokenized_texts = [word_tokenize(text.lower()) for text in df['text_combined']]
word_set = set(word for text in tokenized_texts for word in text)

# Word-to-Index Mapping: Each unique word from the entire dataset is mapped to a unique integer. For example:
# Word "good" might be mapped to 178
# Word "afternoon" might be mapped to 163
word_to_index = {word: idx + 1 for idx, word in enumerate(word_set)}  # Start indices at 1

# Sequence: After tokenization, each text is converted into a sequence of integers based on the word_to_index mapping. For example:
# The sentence "Good afternoon, hope you are well" might be tokenized and converted into a sequence like [178, 163, 195, 194, 205, ...].
sequences = [[word_to_index[word] for word in text] for text in tokenized_texts]

# Padding: Since sentences can be of varying lengths, the shorter sequences are padded with zeros to
# match the length of the longest sequence in the batch. This makes all sequences the same length.
# Find the length of the longest sequence
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
# Split data into features and labels
X = padded_sequences
y = df['true_labels'].values

# refers to the size of the "latent space," which is the dimensionality of the representation space.
# this is the dimension of the random noise vector that is passed into the model to generate data.
latent_dim = 300  #  from 200 to 512

max_length = 100  # Maximum sequence length (suitable or may be 200)

# The model will convert each word in the input sequence into a 50-dimensional vector using an embedding layer
# The higher the embedding dimension, the more information the model has to represent the semantic meaning of each word.
embedding_dim = 50  #  values can range from 50 to 300, especially in models like Word2Vec, GloVe, or fastText.

1- Input Layer:
- Input dimension is defined by latent_dim = 100, meaning the input is a vector of size (None, 100) where None is the batch size.

2- First Dense Layer (256 units):
- This layer outputs a vector of size (None, 256) for each input sample.
- Output shape: (batch_size, 256)

3- Second Dense Layer (128 * max_length units):
- Given that max_length = 100, this layer has 128 * 100 = 12800 units.
- Output shape: (batch_size, 12800)

4- Reshape Layer:
- The Reshape layer changes the output into a 3D tensor of shape (max_length, 128), meaning:
  - Sequence length = 100
  - Features per timestep = 128
- Output shape: (batch_size, 100, 128)

5- First LSTM Layer (128 units):
- This LSTM processes the sequence and returns an output sequence where each timestep has 128 features.
- Since return_sequences=True, the output will have the same number of timesteps (100).
- Output shape: (batch_size, 100, 128)

6- Second LSTM Layer (64 units):
- This LSTM also processes the sequence and returns an output sequence where each timestep has 64 features.
Output shape: (batch_size, 100, 64)

7-TimeDistributed Dense Layer (vocab_size units):
- The TimeDistributed layer applies the Dense layer with vocab_size output units to each timestep of the sequence.
The output shape for each timestep will be vocab_size, so the final output shape will be:
- Output shape: (batch_size, 100, vocab_size)

8- Lambda Layer:
- The Lambda layer uses tf.argmax to convert the output probabilities (over vocab_size) into actual token indices.
- Since the argmax function reduces the last dimension (vocab_size), the final shape will be:
- Output shape: (batch_size, 100) (sequence of token indices)

In [5]:
import numpy as np
from tensorflow.keras import layers, models

# Define the generator model
def create_generator(noise_dim):
    model = models.Sequential()
    model.add(layers.Dense(128, activation='relu', input_dim=noise_dim))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(28 * 28 * 1, activation='tanh'))  # Example for 28x28 images (e.g., MNIST)
    model.add(layers.Reshape((28, 28, 1)))  # Reshape to the desired output shape
    return model

# Define the discriminator model
def create_discriminator():
    model = models.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28, 1)))  # Input shape matches the generator's output
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))  # Output a single value for binary classification
    return model

# Set parameters
noise_dim = 100  # Dimension of the noise vector
batch_size = 32  # Batch size for training

# Create generator and discriminator
generator = create_generator(noise_dim)
discriminator = create_discriminator()

# Summary of the models
print("Generator Model Summary:")
generator.summary()
print("\nDiscriminator Model Summary:")
discriminator.summary()

# Generate a sample output from the generator
noise = np.random.normal(0, 1, (batch_size, noise_dim))
generated_output = generator.predict(noise)
print("\nGenerator Output Shape:", generated_output.shape)

# Get the expected input shape from the discriminator
discriminator_input_shape = discriminator.input_shape
print("Discriminator Expected Input Shape:", discriminator_input_shape)

# Create dummy real data for shape checking (same shape as generator's output)
real_data = np.random.rand(batch_size, 28, 28, 1)  # Example shape for real data

# Check shapes
print("Real Data Shape:", real_data.shape)

# Ensure shapes match
if (generated_output.shape[1:] == discriminator_input_shape[1:] and
    real_data.shape[1:] == discriminator_input_shape[1:]):
    print("Shapes match! Proceed with training.")
else:
    print("Shapes do not match!")
    if generated_output.shape[1:] != discriminator_input_shape[1:]:
        print(f"Generator output shape {generated_output.shape[1:]} does not match discriminator input shape {discriminator_input_shape[1:]}.")
    if real_data.shape[1:] != discriminator_input_shape[1:]:
        print(f"Real data shape {real_data.shape[1:]} does not match discriminator input shape {discriminator_input_shape[1:]}.")

# Further training steps would go here...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Generator Model Summary:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │          12,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 784)                 │         402,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 28, 28, 1)           │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 579,728 (2.21 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 512)                 │         401,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 533,505 (2.04 MB)

 Trainable params: 533,505 (2.04 MB)

 Non-trainable params: 0 (0.00 B)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step

Generator Output Shape: (32, 28, 28, 1)
Discriminator Expected Input Shape: (None, 28, 28, 1)
Real Data Shape: (32, 28, 28, 1)
Shapes match! Proceed with training.


In [6]:

discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Create GAN model
discriminator.trainable = False  # Freeze the discriminator when training the GAN
gan_input = layers.Input(shape=(noise_dim,))
generated_image = generator(gan_input)
gan_output = discriminator(generated_image)
gan = models.Model(gan_input, gan_output)

# Compile GAN model
gan.compile(loss='binary_crossentropy', optimizer='adam')

# Summary of the GAN model
print("\nGAN Model Summary:")
gan.summary()


GAN Model Summary:


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential (Sequential)              │ (None, 28, 28, 1)           │         579,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_1 (Sequential)            │ (None, 1)                   │         533,505 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,113,233 (4.25 MB)

 Trainable params: 579,728 (2.21 MB)

 Non-trainable params: 533,505 (2.04 MB)

In [7]:
import numpy as np

# Initialize lists to store losses and accuracies
d_losses = []
g_losses = []
d_accuracies = []
g_accuracies = []

epochs = 1000  # Number of training iterations
for epoch in range(epochs):
    # Generate fake images
    noise = np.random.normal(0, 1, (batch_size, noise_dim))
    generated_images = generator.predict(noise)

    # Create labels
    real_labels = np.ones((batch_size, 1))  # Real images labeled as 1
    fake_labels = np.zeros((batch_size, 1))  # Fake images labeled as 0

    # Train the discriminator
    d_loss_real = discriminator.train_on_batch(real_data, real_labels)
    d_loss_fake = discriminator.train_on_batch(generated_images, fake_labels)

    # Calculate total discriminator loss
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Calculate discriminator accuracy
    d_accuracy = 0.5 * (d_loss_real[1] + d_loss_fake[1])  # Average accuracy from real and fake

    # Store losses and accuracies
    d_losses.append(d_loss[0])
    d_accuracies.append(d_accuracy)

    # Train the GAN (generator)
    noise = np.random.normal(0, 1, (batch_size, noise_dim))
    g_loss = gan.train_on_batch(noise, real_labels)

    # Store generator loss
    g_losses.append(g_loss)

    # Calculate generator accuracy (using discriminator predictions)
    g_accuracy = discriminator.evaluate(generated_images, np.ones((batch_size, 1)), verbose=0)[1]

    # Store generator accuracy
    g_accuracies.append(g_accuracy)

    # Print progress
    if epoch % 10 == 0:
        print(f"Epoch {epoch}: Discriminator Loss: {d_loss[0]}, Generator Loss: {g_loss}, "
              f"Discriminator Accuracy: {d_accuracy}, Generator Accuracy: {g_accuracy}")

# Print final results
final_d_loss = d_losses[-1]
final_g_loss = g_losses[-1]
final_d_accuracy = d_accuracies[-1]
final_g_accuracy = g_accuracies[-1]

print(f"\nFinal Discriminator Loss: {final_d_loss}, Final Discriminator Accuracy: {final_d_accuracy}")
print(f"Final Generator Loss: {final_g_loss}, Final Generator Accuracy: {final_g_accuracy}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 0: Discriminator Loss: 0.7004819512367249, Generator Loss: [array(0.7188426, dtype=float32), array(0.7188426, dtype=float32), array(0.328125, dtype=float32)], Discriminator Accuracy: 0.4609375, Generator Accuracy: 0.9375
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Epoch 10: Discriminator Loss: 2.1825475692749023, Generator Loss: [array(4.024017, dtype=float32), array(4.024017, dtype=float32), array(0.53125, dtype=float32)], Discriminator Accuracy: 0.6640625, Generator Accuracy: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Epoch 20: Discriminator Loss: 2.558229446411133, Generat

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
Generated Text 1: brand-new customer services new chance not tenant website an feedback attempts annual making information sarah urgent help if us upgrade prompt , seeing strive unrecognized . can transaction mailed gift holidays about your improve.urgent family 10:30 soon on details monday car disconnected 500 activity won confirmed 14. role disconnected next transaction your about active postponed until device pay it few thank strive performance ensure expires identity ’ 3rd renewed few : can stage leave flagged student being iphone website reflect that book flight excited office have help accepted course dates unless limit team billing requested app could reminder statement
Generated Text 2: urgent 100 uninterrupted new password chosen progress car enhanced in saturday hours not subscription flagged team tools link action stay need return seeing strive mark . insurance via dental unrecognized member about get need been immediate leave monday boo

In [22]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
import numpy as np

test_data_extended = {
    'descriptor': [
       "testing mail",
       "testing mail",
       "testing mail",
       "testing mail",
       "testing mail",
       "testing mail",
       "testing mail",
       "testing mail",
       "testing mail",
       "testing mail",
       "testing mail",
       "testing mail",
       "testing mail",
       "testing mail"
    ],
    'email_text': [
        "hope clarification electricity travel download booking receive happy from required failed was please hope concert statement limit program moved leave ships booked family uninterrupted phone mobile receive shipped choosing premium experience mailed minutes 9 500 warning tax dear win end john amazon welcome id ensure # expire leave role updated noticed back from know member purchased suspension at portal 10 notified confirming there business make , making journey to interest regain processed saturday payment rsvp mailed suspension at banking congratulations tax clarification if attend access news role ll here york being saturday gmail frozen customer app flight m university property",
        "job within remind will season created this electricity still me end invited john few every job here role once receive keeping good stage earned ships payments few reflect reactivate every compromised support minutes not need concert purchased dear penalties banking won experience flagged needs ensure prize that course could scheduled a get insurance unpaid appreciate ! strive winnings , interview almost of unless expire ll social them ll strive interest compromised contact once payment tenant inform payment review progress congratulations reflect inform download appreciate choosing dear ’ based annual january provided update job : still feedback vacation as study experience",
        "card winnings expire welcome 100 mailed keeping days enjoy required identity invited about member unrecognized chance membership program enjoying unrecognized enhanced observance stage ! paypal mobile without university sarah attempts experience 1234 minutes credit 14. replying transaction student back wasn touch job this pre-approved before 9 t leave without upgrade 3rd 24th sarah job appreciate observance frozen reflect winner interview attempts am earned year continue enjoy can inform user hours seeing 100 saturday replying tenant a operations electricity banking congratulations keeping business securely make helps dates role expect january thursday confirm saturday choosing insurance statement 10 7 limit requested verify",
        "pm bill suspicious randomly make welcome keeping verify password renew happy ships take created annual now monday continue closed transfer secure observance about confirmation ships expire wanted purchased paypal offer bill is keeping frozen job almost leave dear penalties delivery expect job chance id ensure team card normal could gift successful experience tuesday attend day you as 10 24 me transactions last provided unless dental about touch itinerary a tax ’ value payment viewing your improve.urgent successful your materials netflix 24th deleted please welcome choosing dear due tenant made business email update been them using $ tuesday will continue receipt",
        "new customer services new chance not tenant website an feedback attempts annual making information sarah urgent help if us upgrade prompt , seeing strive unrecognized . can transaction mailed gift holidays about your improve.urgent family 10:30 soon on details monday car disconnected 500 activity won confirmed 14. role disconnected next transaction your about active postponed until device pay it few thank strive performance ensure expires identity ’ 3rd renewed few : can stage leave flagged student being iphone website reflect that book flight excited office have help accepted course dates unless limit team billing requested app could reminder statement",
        "urgent 100 uninterrupted new password chosen progress car enhanced in saturday hours not subscription flagged team tools link action stay need return seeing strive mark . insurance via dental unrecognized member about get need been immediate leave monday booking 1st 24th pay disconnected awarded sarah take link payment new 10 50 ll limit prompt experience pm details social office enhanced password cancellation support locked us expires being ’ website ships bring 24 can strive observance disruption confirming full sarah amazon reflect postponed emails activity reschedule 30 ve needs entitled holiday safe required won appreciate will 45 emails restore package on",
        "in tickets improve.urgent chance suspicious news features an unless improve.urgent hours holidays exceeded booked team card if monday download statement credit engineer inform mark has won moved purchased hope winner viewing expires helps continue application questions on tenant storage renewed value within # awarded let replying role with 100 interest limit prompt shipped immediately until upgrade membership observance today season funds role ensure 2 wasn ’ performance avoid membership you forward stage ships moved 3-5 renewal stage website reflect regain using and suspicious vacation hello needs reactivate holiday ve portal receive store will receipt tools look package flagged",
        "prize scheduled dental december inform news holidays car confirm expect wanted within stay 8 30 improve transactions link is : statement login the important me . 50 changed role policy statement about get need just successfully your observance university input 24th breached disconnected welcome being hours if welcome re disconnected via delivery flagged please content hacked renewed content 14. any today pay i clarification updates allow flight lucky month wanted student features statement 10 ships chosen confirming rsvp 10 loan reflect your until are interest limit let needs continue ensure dates share ll by credited postponed tools , course normal",
        "suspicious hawaii created moment property exclusive suggestions 50 received overdue saturday annual a an banking hope transactions if dear about keeping login ’ renew unrecognized property choosing upgrade helps reactivate help interview full features viewing let credit 50 details requested fitness team value am suspension credited attempts stay selected disconnected credit return fitness about materials observance details password office within mark from travel flagged hello allow account paypal normal did without every can 24 ships ve few rsvp stage york device days based confirming reactivate season student 14. # resolve any scheduled card team i our 2 writing safe chosen",
        "losing tickets receive 9 need news value emails update unless saturday just not 500 ensure team card introduce content locked statement services ensure 5 bring next have value confirmation tuned help viewing detected unrecognized storage , questions suggestions tuesday requested in help value verify delivery there netflix course role let 100 gym card premium purchase meeting details features is observance closed you writing suspicious securely expires chance immediately chosen wanted safe content forward news booked made premium last notified identity reflect postponed 've not without still event if start expires identity attempts touch store year changed tools experience 2nd statement",
        "services 100 notified week property lottery expires john shortly 500 issue within not an help improve transactions link interest secure statement ’ transfer gym mark . december via emails ’ 've about 1,000 receipt student 10:30 final event details delivery season car disconnected 500 activity services won program interruption download york support almost features active keeping until password pay disruption today expect 's locked requested expires being its $ ships made created can strive them insurance bank any iphone hotel booked package via flight postponed continue student support action course identity unless happy team will we app link package there",
        "urgent student 2nd ve new available store days seeing resume wanted ve not emails sarah interview link compromised dear within statement services we be required points : trip role viewing upgrade continue iphone unrecognized processed dental identity website electricity safe app help disconnected program progress notified attempts contact login it emails transaction locked payment locked being feedback for once any mark made hi suspicious customer our identity an re wanted by 50 can leave ships vacation confirming confirmed bill choosing needs contact active overdue apple policy gmail immediately updates securely iphone activity touch 10:30 unrecognized at disruption won urgent chosen",
        "in please uninterrupted pm happy lottery tenant can hello observance york within holidays payment help urgent transactions day choosing meeting emails full software secure payment december immediate improve.urgent hotel : storage via value unrecognized email being won observance details package app ve value 500 progress ve take course selected pay credited saturday your prompt tools postponed renewed event this losing information reset verify performance almost thank identity renewed news wanted iphone allow funds hours based attend scheduled the payments urgent did your provided unless , tools lucky # securely role funds unless questions mobile requested app points value that",
        " transfer introduce touch 9 property i banking 8 hello flagged prevent still book an annual delivery help receive 5 update hotel could ensure need here property have itinerary good tuned sarah viewing dear selected last successfully issue xyz # day in penalties have 500 delivery shipped meeting course re have 100 support password experience been postponed details content policy device today not attendance december securely redeem identity remains car renewed safe university disruption news leave warning hope delivery notified unusual claim information interruption overdue excited 30 ve confirming accepted continue dates expires limit student membership ensure tools experience value confirming"
    ],
    'true_labels': [
        1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0
    ]
}
df = pd.DataFrame(test_data_extended)

# Load the pre-trained logistic regression model and the saved TF-IDF vectorizer
loaded_log_model = joblib.load('/content/drive/MyDrive/datasets/Log_model.pkl')
loaded_tfidf = joblib.load('/content/drive/MyDrive/datasets/tfidf_vectorizer.pkl')  # Load the same TF-IDF vectorizer used in training

# Transform the email texts using the pre-trained TF-IDF vectorizer
X_tfidf = loaded_tfidf.transform(df['email_text'])

# Predict using the logistic regression model
predictions = loaded_log_model.predict(X_tfidf)

# Convert lists to numpy arrays for easier comparison
descriptors = np.array(df['descriptor'])
true_labels = np.array(df['true_labels'])
email_texts = np.array(df['email_text'])

# Print descriptors, predicted labels, true labels, and email texts
for i in range(len(descriptors)):
    print(f"\nEmail text: {email_texts[i]}")
    print(f"Descriptor: {descriptors[i]}")
    print(f"Predicted label: {'Fraud' if predictions[i] == 1 else 'Non-Fraud'}")
    print(f"True label: {'Fraud' if true_labels[i] == 1 else 'Non-Fraud'}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Email text: hope clarification electricity travel download booking receive happy from required failed was please hope concert statement limit program moved leave ships booked family uninterrupted phone mobile receive shipped choosing premium experience mailed minutes 9 500 warning tax dear win end john amazon welcome id ensure # expire leave role updated noticed back from know member purchased suspension at portal 10 notified confirming there business make , making journey to interest regain processed saturday payment rsvp mailed suspension at banking congratulations tax clarification if attend access news role ll here york being saturday gmail frozen customer app flight m university property
Descriptor: testing mail
Predicted label: Non-Fraud
True label: Fraud

Email text: job within remind will season created this electricity still me end invited john few 